
# Assignment 2 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [2]:
# Part 1
import numpy as np
# Part 1
# Transition matrix
transition_matrix = np.array([[0.3, 0.4, 0.3],  # downtown
                              [0.2, 0.5, 0.3],  # suburbs
                              [0.4, 0.3, 0.3]]) # countryside
# Two steps --> transition matrix squared
transition_matrix_2steps = np.dot(transition_matrix, transition_matrix)

# Probability of going in downtown after 2 steps given it started on suburbs
probability = transition_matrix_2steps[1, 0] 
# Fill in the answer to part 1 below as a decimal number
print(transition_matrix_2steps)
problem1_p1 = 0.28

[[0.29 0.41 0.3 ]
 [0.28 0.42 0.3 ]
 [0.3  0.4  0.3 ]]


In [3]:
# Part 2
# Probability of going in downtown for the first time after 2 steps given it started on suburbs
# The available options will be to start from Surburbs/Countryside and go to Surburbs/Countryside and then from there (Surburbs/Countryside)
# you go to Downtown

S_to_S = transition_matrix[1, 1]
S_to_C = transition_matrix[1, 2]
S_to_D = transition_matrix[1, 0]
C_to_D = transition_matrix[2, 0]

# Possibility 1: starts S--> S--> D   
prob1 = S_to_S * S_to_D

# Possibility 1: starts S--> C--> D 
prob2 = S_to_C * C_to_D

# Total probability 
total_prob = prob1+prob2
# Fill in the answer to part 2 below as a decimal number
problem1_p2 = 0.22

In [4]:
# Part 3
# It is true as we can reach any state from any state using any number of steps and the prob will always be > 0
# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [5]:
# Part 4
# States (three states)
n = transition_matrix.shape[0]

# P^T - I
A = np.transpose(transition_matrix) - np.eye(n)

# Append the normalization condition 
A = np.vstack([A, np.ones(n)])

# Create the result vector (stationary conditions=0) 
b = np.zeros(n + 1)
# Create the result vector (normalisation conditions=1)
b[-1] = 1  

# A*π = b to find the stationary distribution
π = np.linalg.solve(A.T @ A, A.T @ b)  # Solve using least squares to account for possible rank deficiency
# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
problem1_stationary = [0.29, 0.41, 0.3]

In [6]:
# Part 5
# We are interested in the surburbs and countryside states so we are seperating their transition probabilities
transition_matrix2 = np.array([[0.5, 0.3], [0.3, 0.3]])

# Identity matrix 
I = np.eye(transition_matrix2.shape[0])
f_matrix = np.linalg.inv(I - transition_matrix2)
# The first row of the table gives us the expected number of steps
expected_steps_from_suburbs = f_matrix[0].sum()
# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
problem1_ET = 3.8
print(expected_steps_from_suburbs)

3.846153846153846


---
## Assignment 2, PROBLEM 2
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  


 **Normal Distribution PDF**
For a random variable \( X \) that follows a **Normal distribution** with parameters \( \mu \) (mean) and \( \sigma \) (standard deviation), the PDF is given by:

$$
f(x; \mu, \sigma) = \frac{1}{\sigma \sqrt{2 \pi}} \exp\left(-\frac{(x - \mu)^2}{2\sigma^2}\right).
$$

 **Likelihood Function**
For \( n \) **independent** samples \( x_1, x_2, \dots, x_n \) drawn from the Normal distribution \( N(\mu, \sigma^2) \), the **likelihood function** \( L(\mu, \sigma) \) is the product of the individual PDFs:

$$ L(\mu, \sigma) = \prod_{i=1}^n f(x_i; \mu, \sigma).
$$

Substitute the PDF $$ f(x; \mu, \sigma) $$ into the likelihood:

$$
L(\mu, \sigma) = \prod_{i=1}^n \left( \frac{1}{\sigma \sqrt{2 \pi}} \exp\left(-\frac{(x_i - \mu)^2}{2 \sigma^2}\right) \right).
$$

**Log Likelihood Function**
The **log likelihood function** is the natural logarithm of the likelihood:

$$
\log L(\mu, \sigma) = \log \left( \prod_{i=1}^n f(x_i; \mu, \sigma) \right).
$$

Using the logarithmic identity $$ \log(ab) = \log(a) + \log(b) $$ this becomes:

$$
\log L(\mu, \sigma) = \sum_{i=1}^n \log \left( \frac{1}{\sigma \sqrt{2 \pi}} \exp\left(-\frac{(x_i - \mu)^2}{2 \sigma^2}\right) \right).
$$

$$
\log L(\mu, \sigma) = \sum_{i=1}^n \left[ -\log(\sigma \sqrt{2 \pi}) - \frac{(x_i - \mu)^2}{2 \sigma^2} \right].
$$

$$
\log L(\mu, \sigma) = -n \log(\sigma \sqrt{2 \pi}) - \sum_{i=1}^n \frac{(x_i - \mu)^2}{2 \sigma^2}.
$$

$$
\log L(\mu, \sigma) = -n \log(\sigma) - n \log(\sqrt{2 \pi}) - \frac{1}{2 \sigma^2} \sum_{i=1}^n (x_i - \mu)^2.
$$
 **Negative Log Likelihood**
To perform optimization (which usually minimizes a function), we minimize the **negative log likelihood**. So, we define the **negative log likelihood** as:

$$
-\log L(\mu, \sigma) = n \log(\sigma) + \frac{n}{2} \log(2 \pi) + \frac{1}{2 \sigma^2} \sum_{i=1}^n (x_i - \mu)^2.
$$

In this example, the initial guess for the mean (μ) is 10 and for the standard deviation (σ) is 1. These initial values are reasonable because the target distribution is known to be N(10,2), where the mean is 10 and the standard deviation is sqrt 2≈1.414.These initial values are within the expected range of the parameter values, making them a good starting point for the optimization.


The purpose of the problem you're working on is to estimate the parameters of a Normal distribution (specifically the mean 
μ and standard deviation σ) based on a set of observed data points. In this case, the data is generated from a Normal distribution with mean 10 and variance 2, but the goal is to estimate these parameters using Maximum Likelihood Estimation (MLE) and optimization techniques.

In [8]:

import numpy as np
from scipy import optimize
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30) 

# define the negative log likelihoo function you want to minimise by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param=parameters[0] # Get the mu
    sigma_param=parameters[1] # Get the sigma 
    if sigma_param <= 0:
        return np.inf  
    n = len(normallySimulatedDataSamples) # Get the sample size
    # Get the log likelihood
    log_likelihood = ((n / 2) * np.log(2 * np.pi) + n * np.log(sigma_param) + (1 / (2 * sigma_param**2)) * np.sum((normallySimulatedDataSamples - mu_param)**2))
    return log_likelihood 

# you should only change XXX below and not anything else
parameter_bounding_box=((-20, 20), (0.1, 5.0)) # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([10, 1]) # point in 2D to initialise the minimize algorithm
result_problem2_opt = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box,  method='L-BFGS-B') 
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_problem2_opt


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 58.63138728292264
        x: [ 9.269e+00  1.708e+00]
      nit: 9
      jac: [-3.055e-05  1.457e-04]
     nfev: 36
     njev: 12
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

fun: 58.63138728292264:
This is the value of the objective function (negative log-likelihood) at the optimal point. The objective function value is 58.63 in this case, which represents the minimum negative log-likelihood.
x: [ 9.269e+00 1.708e+00]:
These are the estimated values of the parameters that minimize the objective function. The result [ 9.269e+00, 1.708e+00] corresponds to:
μ=9.269 (estimated mean) σ=1.708 (estimated standard deviation)

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

### Given PDF:
The probability density function for the random variable \( X \) is given by:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \quad \lambda > 0, \quad x > 0
$$

This is a parametric form of the probability distribution where \( \lambda \) is the parameter to estimate. The task is to find the value of \( \lambda \) that maximizes the likelihood of observing the given data.

Likelihood Function
For a set of \( n \) **independent and identically distributed (i.i.d.)** samples \( x_1, x_2, \dots, x_n \), the likelihood function \( L(\lambda) \) is the product of the PDFs for each observation:

$$
L(\lambda) = \prod_{i=1}^{n} f(x_i; \lambda)
$$

$$
L(\lambda) = \prod_{i=1}^{n} \left( \frac{1}{24} \lambda^5 x_i^4 \exp(-\lambda x_i) \right)
$$

$$
L(\lambda) = \left( \frac{1}{24} \right)^n \lambda^{5n} \left( \prod_{i=1}^{n} x_i^4 \right) \exp\left( -\lambda \sum_{i=1}^{n} x_i \right)
$$

Log-Likelihood Function
$$
\ell(\lambda) = \log(L(\lambda)) = n \log\left( \frac{1}{24} \right) + 5n \log(\lambda) + 4 \sum_{i=1}^{n} \log(x_i) - \lambda \sum_{i=1}^{n} x_i
$$

Since the terms involving \( x_i \) are independent of \( \lambda \), we can ignore them for the purpose of maximization:

$$
\ell(\lambda) = 5n \log(\lambda) - \lambda \sum_{i=1}^{n} x_i + \text{(constant terms with respect to } \lambda)
$$

 Maximization
Now, we differentiate the log-likelihood function with respect to \( \lambda \) and set it to 0 to find the maximum likelihood estimate for \( \lambda \):

$$
\frac{d\ell(\lambda)}{d\lambda} = \frac{5n}{\lambda} - \sum_{i=1}^{n} x_i = 0
$$

$$
\lambda = \frac{5n}{\sum_{i=1}^{n} x_i}
$$

 **Maximum Likelihood Estimate (MLE)

$$
\hat{\lambda} = \frac{5n}{\sum_{i=1}^{n} x_i}
$$



In [9]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''write comment of what this function does'''
    # Number of samples given by the user 
    n = len(x)  
    # Get the sum of all the samples 
    sum_of_x = sum(x)  
    # The MLE of lambda is given by the following formula
    lambda_MLe = 5*n/sum_of_x 
    return lambda_MLe

---
## Assignment 2, PROBLEM 4
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [8]:

def problem4_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    # Parameters for the LCG. These numbers satisfy the Hull-Dobell Theorem)
    M = 2**31  
    a = 1103515245 
    b = 12345 
    
    # Start at seed=0
    u = seed
    generated_values = []
    # Use the formula to generate the numbers
    for _ in range(size):
        u = (a * u + b) % M 
        generated_values.append(u / M) 
    
    return generated_values

In [9]:

def problem4_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    # Generate random numbers
    random_values = generator(size=size, seed=seed)
    
    # Normalising
    uniform_sample = np.array(random_values) / period
    
    return uniform_sample

In [10]:

def problem4_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    # Target density function f(x)
    def f_(x):
        return (np.pi / 2) * abs(np.sin(2 * np.pi * x))
    # f(x) ≤ M * g(x) 
    M = np.pi/2
    np.random.seed(seed)  
    accepted_samples = []
    while len(accepted_samples) < n_iterations:
        # Generate a candidate x from g(x)
        x = uniformGenerator(size=1, seed=np.random.randint(0, 10000))[0]
        # Find r(x) = f(x)/M
        r_x = f_(x)/M
        # Generate u
        u = uniformGenerator(size=1, seed=np.random.randint(0, 10000))[0]
        # Accept only samples that are smaller than the calculated ratio
        if u <= r_x:
            accepted_samples.append(x)
    
    return accepted_samples


$$
f(x) = \frac{\pi}{2} |\sin(2 \pi x)|
$$

Since \( |\sin(2\pi x)| \) is always non-negative, we can rewrite it without the absolute value for simplicity, keeping in mind that \( \sin(2\pi x) \) alternates in sign. So, the function becomes:

$$
f(x) = \frac{\pi}{2} \sin(2\pi x) \quad \text{for } 0 \leq x \leq 1
$$

However, \( |\sin(2\pi x)| \) is symmetric and periodic. Therefore, it suffices to look at one period of \( \sin(2\pi x) \) and its behavior within \( [0, 1] \).

### Find the Derivative

Now, to find the maximum of \( f(x) \), we need to compute its derivative. The derivative of \( f(x) = \frac{\pi}{2} \sin(2\pi x) \) is:

$$
f'(x) = \frac{\pi}{2} \cdot 2\pi \cos(2\pi x)
$$

$$
f'(x) = \pi^2 \cos(2\pi x)
$$

### Find Critical Points

To find the critical points, we set the derivative equal to zero:

$$
\pi^2 \cos(2\pi x) = 0
$$
$$
\cos(2\pi x) = 0
$$

The solutions to $$ \cos(2\pi x) = 0 $$ are:

$$
2\pi x = \frac{\pi}{2} + n\pi \quad \text{for} \quad n \in \mathbb{Z}
$$

$$
x = \frac{1}{4} + \frac{n}{2}
$$

Thus, within the interval \( [0, 1] \), the critical points are:

$$
x = \frac{1}{4}, \frac{3}{4}
$$

### Determine the Maximum

To determine whether these critical points correspond to maxima or minima, we look at the second derivative:

$$
f''(x) = -\pi^3 \sin(2\pi x)
$$

Evaluating the second derivative at the critical points:

- At $$ x = \frac{1}{4} $$ $$ \sin(2\pi x) = \sin\left( \frac{\pi}{2} \right) = 1 $$ so $$ f''(x) = -\pi^3 $$ which is negative, indicating a local maximum at $$ x = \frac{1}{4} $$
- At $$ x = \frac{3}{4} $$ $$ \sin(2\pi x) = \sin\left( \frac{3\pi}{2} \right) = -1 $$ so $$ f''(x) = \pi^3 $$ which is positive, indicating a local minimum at $$ x = \frac{3}{4} $$

Thus, the maximum of \( f(x) \) occurs at $$ x = \frac{1}{4} $$

###  Maximum Value

Now, let's compute the maximum value of \( f(x) \) at $$ x = \frac{1}{4} $$

$$
f\left( \frac{1}{4} \right) = \frac{\pi}{2} |\sin(2 \pi \cdot \frac{1}{4})|
$$

Since \( \sin\left( \frac{\pi}{2} \right) = 1 \), we get:

$$
f\left( \frac{1}{4} \right) = \frac{\pi}{2}
$$

Thus, the maximum value of the function \( f(x) \) is \( \frac{\pi}{2} \), and the maximum occurs at \( x = \frac{1}{4} \).



---
#### Local Test for Assignment 2, PROBLEM 4
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [13]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem4_LCG(size=10, seed = 1))

period = 2**31 

print("Uniform sampler %s" % problem4_uniform(generator=problem4_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem4_uniform(generator=problem4_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [0.5138700781390071, 0.1757413032464683, 0.3086515162140131, 0.5345338867045939, 0.9476279253140092, 0.17173630138859153, 0.7022311687469482, 0.2264306810684502, 0.49477344658225775, 0.12472031963989139]
Uniform sampler [2.39289402e-10 8.18359215e-11 1.43727062e-10 2.48911738e-10
 4.41273640e-10 7.99709472e-11 3.27001870e-10 1.05440002e-10
 2.30396840e-10 5.80774246e-11]
Accept-Reject sampler [np.float64(4.575016799863302e-10), np.float64(3.76418638012993e-10), np.float64(1.5006204482151397e-10), np.float64(3.1739517958317975e-10), np.float64(3.7557725310853607e-10), np.float64(1.313680008958179e-10), np.float64(1.9076176554187751e-10), np.float64(3.4919200365447767e-10), np.float64(1.7234818325100043e-10), np.float64(2.873998667090455e-10), np.float64(8.353799791673e-11), np.float64(2.5828187657193913e-10), np.float64(3.5392391361382947e-10), np.float64(3.0968600513853195e-10), np.float64(2.7646890397685364e-10), np.float64(3.385257699162203e-11), np.float64(3.428851373839

In [14]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.8557786762749672, 0.20168471862593207, 0.6609328098051321, 0.7534589225609072, 0.3940937135353847, 0.0926015922180351, 0.14708406988922706, 0.2065378785625105, 0.7578311452361404, 0.244386414415232, 0.22601718040867969, 0.31545685877712215, 0.7384157293035217, 0.7238293963133157, 0.8048047269727561, 0.7582160220917413, 0.08275869189890106, 0.32178919106178483, 0.11798029153147327, 0.19562843006143416]
